In [88]:
import pickle
import numpy,sklearn,pandas
import numpy as np
import pandas as pd

# Reading the dataset file

In [89]:
raw_data = pd.read_csv("C:\\Users\\AKASH\\Desktop\\dataset2.csv") 

In [90]:
raw_data.head()

,URL,Target
0,https://locking-app-adverds.000webhostapp.com/...,yes
1,http://www.myhealthcarepharmacy.ca/wp-includes...,yes
2,http://code.google.com/p/pylevenshtein/,no
3,http://linkedin.com/,no
4,http://imageshack.com/f/219/cadir2yr3.jpg,no


In [91]:
raw_data.shape

(1781, 2)

# SPLITTING THE URL

In [93]:
raw_data['URL'].str.split("://").head()

0    [https, locking-app-adverds.000webhostapp.com/...
1    [http, www.myhealthcarepharmacy.ca/wp-includes...
2             [http, code.google.com/p/pylevenshtein/]
3                                [http, linkedin.com/]
4           [http, imageshack.com/f/219/cadir2yr3.jpg]
Name: URL, dtype: object

In [94]:
seperation_of_protocol = raw_data['URL'].str.split("://",expand = True) #expand argument in the split method will give you a new column

In [95]:
seperation_of_protocol.head()

,0,1,2,3,4,5,6
0,https,locking-app-adverds.000webhostapp.com/payment-...,None,None,None,None,None
1,http,www.myhealthcarepharmacy.ca/wp-includes/js/jqu...,None,None,None,None,None
2,http,code.google.com/p/pylevenshtein/,None,None,None,None,None
3,http,linkedin.com/,None,None,None,None,None
4,http,imageshack.com/f/219/cadir2yr3.jpg,None,None,None,None,None


In [96]:
type(seperation_of_protocol)

pandas.core.frame.DataFrame

In [97]:
seperation_domain_name = seperation_of_protocol[1].str.split("/",1,expand = True) #split(seperator,no of splits according to seperator(delimiter),expand)


In [98]:
type(seperation_domain_name)

pandas.core.frame.DataFrame

In [99]:
seperation_domain_name.columns=["domain_name","address"] #renaming columns of data frame

In [100]:
seperation_domain_name.head()

,domain_name,address
0,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...
1,www.myhealthcarepharmacy.ca,wp-includes/js/jquery/ini.php
2,code.google.com,p/pylevenshtein/
3,linkedin.com,
4,imageshack.com,f/219/cadir2yr3.jpg


In [101]:
splitted_data = pd.concat([seperation_of_protocol[0],seperation_domain_name],axis=1)

In [102]:
splitted_data.columns = ['protocol','domain_name','address']

In [103]:
splitted_data.head()

,protocol,domain_name,address
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...
1,http,www.myhealthcarepharmacy.ca,wp-includes/js/jquery/ini.php
2,http,code.google.com,p/pylevenshtein/
3,http,linkedin.com,
4,http,imageshack.com,f/219/cadir2yr3.jpg


In [104]:
splitted_data['is_phished'] = pd.Series(raw_data['Target'], index=splitted_data.index)

In [105]:
splitted_data

,protocol,domain_name,address,is_phished
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes
1,http,www.myhealthcarepharmacy.ca,wp-includes/js/jquery/ini.php,yes
2,http,code.google.com,p/pylevenshtein/,no
3,http,linkedin.com,,no
4,http,imageshack.com,f/219/cadir2yr3.jpg,no
...,...,...,...,...
1776,https,docs.google.com,document/u/1/,no
1777,http,www.charlestodd.com,wp-includes/pomo/adobe2.html,yes
1778,http,tslimpact.com,medsynaptic/wp-content/themes/twentyseventeen/...,yes
1779,http,daoudilorin11.mystagingwebsite.com,wp-content/plugins/ubh/acc/dir/68e1c/dir/car.php,yes


In [106]:
type(splitted_data)

pandas.core.frame.DataFrame

# Features Extraction
Feature-1

1.Long URL to Hide the Suspicious Part

If the length of the URL is greater than or equal 54 characters then the URL classified as phishing

0 --- indicates legitimate

1 --- indicates Phishing

2 --- indicates Suspicious

In [107]:
def long_url(l):
    l= str(l)
    """This function is defined in order to differntiate website based on the length of the URL"""
    if len(l) < 54:
        return 0
    elif len(l) >= 54 and len(l) <= 75:
        return 2
    return 1

In [108]:
splitted_data['long_url'] = raw_data['URL'].apply(long_url) 

In [109]:
splitted_data[splitted_data.long_url == 0] 

,protocol,domain_name,address,is_phished,long_url
2,http,code.google.com,p/pylevenshtein/,no,0
3,http,linkedin.com,,no,0
4,http,imageshack.com,f/219/cadir2yr3.jpg,no,0
6,http,www.7-zip.org,download.html,no,0
7,http,ebay.com,,no,0
...,...,...,...,...,...
1766,http,malomolk.com,nab/cardinfo.html,yes,0
1774,http,www.dpincsupport.com,,no,0
1775,https,bitcoin.org,en/,no,0
1776,https,docs.google.com,document/u/1/,no,0


# Feature-2

2.URL’s having “@” Symbol

Using “@” symbol in the URL leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol.

IF {Url Having @ Symbol→ Phishing Otherwise→ Legitimate }

0 --- indicates legitimate

1 --- indicates Phishing

In [110]:
def have_at_symbol(l):
    """This function is used to check whether the URL contains @ symbol or not"""
    if "@" in str(l):
        return 1
    return 0

In [111]:
splitted_data['having_@_symbol'] = raw_data['URL'].apply(have_at_symbol)

In [112]:
splitted_data

,protocol,domain_name,address,is_phished,long_url,having_@_symbol
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes,1,0
1,http,www.myhealthcarepharmacy.ca,wp-includes/js/jquery/ini.php,yes,2,0
2,http,code.google.com,p/pylevenshtein/,no,0,0
3,http,linkedin.com,,no,0,0
4,http,imageshack.com,f/219/cadir2yr3.jpg,no,0,0
...,...,...,...,...,...,...
1776,https,docs.google.com,document/u/1/,no,0,0
1777,http,www.charlestodd.com,wp-includes/pomo/adobe2.html,yes,2,0
1778,http,tslimpact.com,medsynaptic/wp-content/themes/twentyseventeen/...,yes,1,0
1779,http,daoudilorin11.mystagingwebsite.com,wp-content/plugins/ubh/acc/dir/68e1c/dir/car.php,yes,1,0


# Feature-3

3.Redirecting using “//”

The existence of “//” within the URL path means that the user will be redirected to another website. An example of such URL’s is: “http://www.legitimate.com//http://www.phishing.com”. We examine the location where the “//” appears. We find that if the URL starts with “HTTP”, that means the “//” should appear in the sixth position. However, if the URL employs “HTTPS” then the “//” should appear in seventh position.

IF {ThePosition of the Last Occurrence of "//" in the URL > 7→ Phishing

Otherwise→ Legitimate
0 --- indicates legitimate

1 --- indicates Phishing

In [113]:
def redirection(l):
    """If the url has symbol(//) after protocol then such URL is to be classified as phishing """
    if "//" in str(l):
        return 1
    return 0

In [114]:
splitted_data['redirection_//_symbol'] = seperation_of_protocol[1].apply(redirection)

In [115]:
splitted_data.head()

,protocol,domain_name,address,is_phished,long_url,having_@_symbol,redirection_//_symbol
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes,1,0,0
1,http,www.myhealthcarepharmacy.ca,wp-includes/js/jquery/ini.php,yes,2,0,0
2,http,code.google.com,p/pylevenshtein/,no,0,0,0
3,http,linkedin.com,,no,0,0,0
4,http,imageshack.com,f/219/cadir2yr3.jpg,no,0,0,0


# Feature-4

4.Adding Prefix or Suffix Separated by (-) to the Domain

The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name so that users feel that they are dealing with a legitimate webpage.

For example http://www.Confirme-paypal.com/.

IF {Domain Name Part Includes (−) Symbol → Phishing

Otherwise → Legitimate
1 --> indicates phishing

0 --> indicates legitimate

In [116]:
def prefix_suffix_seperation(l):
    if '-' in str(l):
        return 1
    return 0

In [117]:
splitted_data['prefix_suffix_seperation'] = seperation_domain_name['domain_name'].apply(prefix_suffix_seperation)

In [118]:
splitted_data.head()

,protocol,domain_name,address,is_phished,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes,1,0,0,1
1,http,www.myhealthcarepharmacy.ca,wp-includes/js/jquery/ini.php,yes,2,0,0,0
2,http,code.google.com,p/pylevenshtein/,no,0,0,0,0
3,http,linkedin.com,,no,0,0,0,0
4,http,imageshack.com,f/219/cadir2yr3.jpg,no,0,0,0,0


# Feature - 5

Sub-Domain and Multi Sub-Domains
The legitimate URL link has two dots in the URL since we can ignore typing “www.”. If the number of dots is equal to three then the URL is classified as “Suspicious” since it has one sub-domain. However, if the dots are greater than three it is classified as “Phishy” since it will have multiple sub-domains

0 --- indicates legitimate

1 --- indicates Phishing

2 --- indicates Suspicious



In [119]:
def sub_domains(l):
    l= str(l)
    if l.count('.') < 3:
        return 0
    elif l.count('.') == 3:
        return 2
    return 1

In [120]:
splitted_data['sub_domains'] = splitted_data['domain_name'].apply(sub_domains)

In [121]:
splitted_data

,protocol,domain_name,address,is_phished,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes,1,0,0,1,0
1,http,www.myhealthcarepharmacy.ca,wp-includes/js/jquery/ini.php,yes,2,0,0,0,0
2,http,code.google.com,p/pylevenshtein/,no,0,0,0,0,0
3,http,linkedin.com,,no,0,0,0,0,0
4,http,imageshack.com,f/219/cadir2yr3.jpg,no,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1776,https,docs.google.com,document/u/1/,no,0,0,0,0,0
1777,http,www.charlestodd.com,wp-includes/pomo/adobe2.html,yes,2,0,0,0,0
1778,http,tslimpact.com,medsynaptic/wp-content/themes/twentyseventeen/...,yes,1,0,0,0,0
1779,http,daoudilorin11.mystagingwebsite.com,wp-content/plugins/ubh/acc/dir/68e1c/dir/car.php,yes,1,0,0,0,0


# Classification of URLs using BAGGING CLASSIFIER

In [124]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import confusion_matrix, accuracy_score

In [125]:
#Features
x = splitted_data.columns[4:9]
x   

Index(['long_url', 'having_@_symbol', 'redirection_//_symbol',
       'prefix_suffix_seperation', 'sub_domains'],
      dtype='object')

In [126]:
#variable to be predicted; yes = 0 and no = 1
y = pd.factorize(splitted_data['is_phished'])[0]
y 

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

# BAGGING CLASSIFIER
A Bagging classifier is an ensemble meta-estimator that fits base classifiers each on random subsets of the original dataset and then aggregate their individual predictions (either by voting or by averaging) to form a final prediction. Such a meta-estimator can typically be used as a way to reduce the variance of a black-box estimator (e.g., a decision tree), by introducing randomization into its construction procedure and then making an ensemble out of it.

In [127]:
# Create a random forest Classifier. By convention, clf means 'Classifier'
clf = BaggingClassifier(n_estimators=100)


# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(splitted_data[x], y)



BaggingClassifier(n_estimators=100)

# Extremely Randomized Trees Classifier(Extra Trees Classifier)
It is a type of ensemble learning technique which aggregates the results of multiple de-correlated decision trees collected in a “forest” to output it’s classification result. In concept, it is very similar to a Random Forest Classifier and only differs from it in the manner of construction of the decision trees in the forest. 

In [128]:
gbc=ExtraTreesClassifier(n_estimators=10, max_depth=None,
...     min_samples_split=2, random_state=0)
gbc.fit(splitted_data[x], y)

ExtraTreesClassifier(n_estimators=10, random_state=0)

# LOGISTIC REGRESSION
Logistic regression is a process of modeling the probability of a discrete outcome given an input variable. The most common logistic regression models a binary outcome; something that can take two values such as true/false, yes/no, and so on.

In [129]:
lrm = LogisticRegression(solver='liblinear',penalty='l1',max_iter=100, dual=False)
lrm.fit(splitted_data[x], y)

LogisticRegression(penalty='l1', solver='liblinear')

In [130]:
test_data = pd.read_csv("C:\\Users\\AKASH\\Desktop\\dataset3.csv") 

In [131]:
clf.predict(test_data[x]) #testing the classifier on test data.

array([0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,

In [132]:
gbc.predict(test_data[x]) #testing the classifier on test data.

array([0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,

In [133]:
lrm.predict(test_data[x]) #testing the classifier on test data.

array([0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,

In [134]:
clf.predict_proba(test_data[x])[0:] #predicted probability for each class from random forest.

array([[0.        , 0.88389456, 0.11610544],
       [0.        , 0.50512098, 0.49487902],
       [0.00166678, 0.32373504, 0.67459818],
       ...,
       [0.00166678, 0.32373504, 0.67459818],
       [0.        , 0.39409873, 0.60590127],
       [0.00166678, 0.32373504, 0.67459818]])

In [135]:
gbc.predict_proba(test_data[x])[0:] #predicted probability for each class from gradient boosting.

array([[0.        , 0.88135593, 0.11864407],
       [0.        , 0.50614754, 0.49385246],
       [0.00153846, 0.32153846, 0.67692308],
       ...,
       [0.00153846, 0.32153846, 0.67692308],
       [0.        , 0.38636364, 0.61363636],
       [0.00153846, 0.32153846, 0.67692308]])

In [136]:
lrm.predict_proba(test_data[x])[0:] #predicted probability for each class from LOGISTIC REGRESSION.

array([[0.00091681, 0.84419439, 0.1548888 ],
       [0.00091733, 0.45828133, 0.54080135],
       [0.0015806 , 0.34618037, 0.65223904],
       ...,
       [0.0015806 , 0.34618037, 0.65223904],
       [0.00157986, 0.34273585, 0.65568429],
       [0.0015806 , 0.34618037, 0.65223904]])

In [137]:
preds = test_data.is_phished[clf.predict(test_data[x])] #predicted values from random forest

In [138]:
preds2 = test_data.is_phished[gbc.predict(test_data[x])] #predicted values from gradient boosting

In [139]:
preds3 = test_data.is_phished[lrm.predict(test_data[x])] #predicted values from LOGISTIC REGRESSION

In [140]:
preds.head(10)

0    yes
0    yes
1     no
0    yes
1     no
1     no
0    yes
1     no
1     no
1     no
Name: is_phished, dtype: object

In [141]:
preds2.head(10)

0    yes
0    yes
1     no
0    yes
1     no
1     no
0    yes
1     no
1     no
1     no
Name: is_phished, dtype: object

In [142]:
preds3.head(10)

0    yes
1     no
1     no
1     no
1     no
1     no
0    yes
1     no
1     no
1     no
Name: is_phished, dtype: object

In [143]:
actual = pd.Series(test_data['is_phished']) #actual values

In [144]:
confusion_matrix(actual,preds) 

array([[ 94,  80],
       [ 64, 152]], dtype=int64)

In [145]:
confusion_matrix(actual,preds2) 

array([[ 92,  82],
       [ 60, 156]], dtype=int64)

In [146]:
confusion_matrix(actual,preds3) 

array([[139,  35],
       [133,  83]], dtype=int64)

# ACCURACY OF THE BAGGING CLASSISFIER


In [147]:
accuracy_score(actual,preds)*100 #accuracy of bagging classifier

63.07692307692307

# Accuracy Of Extremely Randomized Trees Classifier(Extra Trees Classifier)

In [148]:
accuracy_score(actual,preds2)*100 #accuracy of classifier extra tree classifier

63.589743589743584

# Accuracy Of logistic regression Classifier

In [149]:
accuracy_score(actual,preds3)*100 #accuracy of classifier logistic regression

56.92307692307692

# CONCLUSION
              Based on all the observation from the 6 classification algorithm we found that our model is performing well on the extra tree classifier.